In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import math
from timeit import default_timer as timer
from datetime import datetime, timedelta
import numba

In [2]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500

In [3]:
all_df = pd.read_pickle('../data/ml-ta-all-data-20190416.pkl.gz', compression='gzip')
print('Data loaded')
print(all_df.shape)

Data loaded
(15780, 1243)


In [50]:
# all_df.describe()
all_df.to_csv('../data/ml-ta-all-data-20190416.csv')

In [4]:
numeric_cols = []

for col in all_df.columns:
    if all_df[col].dtype.name in ['object', 'category']:
        print(col, all_df[col].dtype.name)
    elif all_df[col].dtype.name != 'int8':
        numeric_cols.append(col)

print(numeric_cols)

one_week_bollinger_prediction category
two_week_bollinger_prediction category
four_week_bollinger_prediction category
eight_week_bollinger_prediction category
twelve_week_bollinger_prediction category
twenty_six_week_bollinger_prediction category
fifty_two_week_bollinger_prediction category
one_week_bollinger_type category
two_week_bollinger_type category
four_week_bollinger_type category
eight_week_bollinger_type category
twelve_week_bollinger_type category
twenty_six_week_bollinger_type category
fifty_two_week_bollinger_type category
symbol category
GICSSector category
GICSIndustryGroup category
GICSIndustry category
['lastTradePriceOnly', 'adjustedPrice', 'volume', 'previousClose', 'change', 'changeInPercent', '52WeekHigh', '52WeekLow', 'changeFrom52WeekHigh', 'changeFrom52WeekLow', 'percebtChangeFrom52WeekHigh', 'percentChangeFrom52WeekLow', 'allordpreviousclose', 'allordchange', 'allorddayshigh', 'allorddayslow', 'allordpercebtChangeFrom52WeekHigh', 'allordpercentChangeFrom52WeekL

In [5]:
# col defs
from processing_constants import BOLLINGER_PREDICTION_COLUMNS, BOLLINGER_VALUE_COLUMNS

target_col = 'target'

all_df[target_col]

week_starting
2014-12-07       7.332172
2015-03-15      -0.279742
2015-05-17       2.480104
2015-05-31       4.263708
2015-06-28       1.888758
2015-07-19      -1.732065
2015-07-26      -5.068851
2015-11-15      -6.340858
2016-03-27       4.258489
2016-04-10       7.583841
2016-05-08       0.238954
2016-06-12      -4.186852
2016-06-19      -1.558571
2016-07-31      -4.379785
2016-12-11       1.358148
2016-12-25      -1.792700
2017-01-01      -1.520339
2017-01-08      -0.642590
2017-04-23       4.210182
2017-06-04      -3.279408
2017-07-02      -2.591387
2017-07-30       0.841159
2017-08-06       2.741427
2017-09-17       8.236786
2018-01-14      -1.085181
2018-03-18       3.399797
2007-07-08     -28.787882
2007-08-05       8.214287
2007-08-19      41.999996
2007-08-26      39.819000
2007-09-30     -15.181520
2008-01-06       0.476951
2008-03-30      24.637684
2008-06-01     -33.333328
2008-08-24     -76.401871
2008-09-07     -81.081078
2008-10-12     -86.046509
2009-01-04     -42.22221

In [6]:
# Train encoder
from sklearn.preprocessing import LabelEncoder 

symbol_encoder = LabelEncoder()
# need to keep symbol for dividing up data later - so make copy of symbol for tranforming into a numeric
all_df['symbolc'] = all_df['symbol']
all_df['symbolc'] = symbol_encoder.fit_transform(all_df['symbolc'].values)

GICSSector_encoder = LabelEncoder()
all_df['GICSSector'] = GICSSector_encoder.fit_transform(all_df['GICSSector'].values)

GICSIndustryGroup_encoder = LabelEncoder()
all_df['GICSIndustryGroup'] = GICSIndustryGroup_encoder.fit_transform(all_df['GICSIndustryGroup'].values)

GICSIndustry_encoder = LabelEncoder()
all_df['GICSIndustry'] = GICSIndustry_encoder.fit_transform(all_df['GICSIndustry'].values)
    

In [7]:
# Execute one hot encoding for bolinger vals

BOLLINGER_PREDICTION_CATEGORIES = ['Steady', 'Rising', 'Falling']
BOLLINGER_VALUE_CATEGORIES = ['Within', 'Below', 'Above']


def one_hot_encode_field(df, column_name, categories):
    new_cols = pd.get_dummies(df[column_name])
    new_cols = new_cols.astype('int8', errors='ignore')

    new_cols.T.reindex(categories).T.fillna(0)
    
    name_map = {}
    # rename the categories
    for val in categories:
        name_map[val] = column_name + '_' + val

    new_cols.rename(name_map, axis=1, inplace=True)
    
    # Remove the original column
    df.drop([column_name], axis=1, inplace=True)
    
    # Return df with new cols 
    return pd.concat([df, new_cols], axis=1)

for col in BOLLINGER_PREDICTION_COLUMNS:
    all_df = one_hot_encode_field(all_df, col, BOLLINGER_PREDICTION_CATEGORIES)

for col in BOLLINGER_VALUE_COLUMNS:
    all_df = one_hot_encode_field(all_df, col, BOLLINGER_VALUE_CATEGORIES)
    

In [8]:
# Apply standard scaler
from sklearn import preprocessing

# Scale all numeric columns excluding target col
scale_cols = []

for col in all_df.columns:
    if col in numeric_cols and col != target_col:
        scale_cols.append(col)
        
print('Scaling', len(scale_cols), 'of', all_df.shape[1], 'cols')

scaler = preprocessing.RobustScaler()

all_df[scale_cols] = scaler.fit_transform(all_df[scale_cols])


Scaling 1008 of 1272 cols


/Users/neilkloot/anaconda/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:959: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanmedian1d, axis, a, overwrite_input)
/Users/neilkloot/anaconda/lib/python3.6/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [9]:
# Fix NaNs

all_df[scale_cols] = all_df[scale_cols].fillna(0)


In [10]:
all_df.isnull().sum()

lastTradePriceOnly                              0
adjustedPrice                                   0
volume                                          0
previousClose                                   0
change                                          0
changeInPercent                                 0
52WeekHigh                                      0
52WeekLow                                       0
changeFrom52WeekHigh                            0
changeFrom52WeekLow                             0
percebtChangeFrom52WeekHigh                     0
percentChangeFrom52WeekLow                      0
allordpreviousclose                             0
allordchange                                    0
allorddayshigh                                  0
allorddayslow                                   0
allordpercebtChangeFrom52WeekHigh               0
allordpercentChangeFrom52WeekLow                0
asxpreviousclose                                0
asxchange                                       0


In [ ]:
for col in all_df.columns:
    print(col)

In [11]:
from numpy import array

# input and output time steps 
n_input = 16
n_out = 1

rnn_list_x = list()
rnn_list_y = list()
rnn_list_symbols = list()

rnn_list_test_x = list()
rnn_list_test_y = list()
rnn_list_test_symbols = list()

symbols = all_df['symbol'].unique()

for symbol in symbols:
    # Filter to model data for this symbol and re-set the pandas indexes
    model_data = all_df.loc[all_df['symbol'] == symbol]
    # Create target data set
    model_target_data = model_data[target_col]
    # Remove cols not in X array
    model_data.drop(['symbol', target_col], axis=1, inplace=True)
    
    X = list()
    y = list()
    test_X = list()
    test_y = list()
        
    # nuber of features
    n_features = model_data.shape[1]
#     print(n_features)
    
    in_start = 0
    eighty_num = int((len(model_data) - n_input) * .80)
    
    # step over the first 80% of the history one time step at a time - training
    for _ in range(eighty_num):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        # ensure we have enough data for this instance
        if out_end < len(model_data):
            X.append(model_data.iloc[in_start:in_end, :].values)
            y.append(model_target_data.iloc[in_end:out_end].values)
            
            rnn_list_x.append(model_data.iloc[in_start:in_end, :].values)
            rnn_list_y.append(model_target_data.iloc[in_end:out_end].values)
            rnn_list_symbols.append(symbol)
            

        # move along one time step
        in_start += 1

    # step over the last 20% of the history one time step at a time - test
    for _ in range(eighty_num + 1, len(model_data)):
        # define the end of the input sequence
        in_end = in_start + n_input
        out_end = in_end + n_out
        
        # ensure we have enough data for this instance
        if out_end < len(model_data):
            test_X.append(model_data.iloc[in_start:in_end, :].values)
            test_y.append(model_target_data.iloc[in_end:out_end].values)
            
            rnn_list_test_x.append(model_data.iloc[in_start:in_end, :].values)
            rnn_list_test_y.append(model_target_data.iloc[in_end:out_end].values)
            rnn_list_test_symbols.append(symbol)
            

        # move along one time step
        in_start += 1
        
    train_x = array(X)
    train_y = array(y)
    test_x = array(test_X)
    test_y = array(test_y)
    
    print('-' * 80)
    print('Symbol:', symbol)
    print(' - train sequence shape:', train_x.shape)
    print(' - test sequence shape:', test_x.shape)
    
    if len(train_x) > 0 and len(test_x) > 0:
        n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

        # reshape output into [samples, timesteps, features]
        train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))

        print('train n_timesteps', n_timesteps)
        print('train n_features', n_features)
        print('train n_outputs', n_outputs)
        print('-' * 5)

        test_n_timesteps, test_n_features, test_n_outputs = test_x.shape[1], test_x.shape[2], test_y.shape[1]

        # reshape output into [samples, timesteps, features]
        test_y = test_y.reshape((test_y.shape[0], test_y.shape[1], 1))

        print('test n_timesteps', test_n_timesteps)
        print('test n_features', test_n_features)
        print('test n_outputs', test_n_outputs)
        print('-' * 5)

        print('train x rnn len:', len(rnn_list_x))
        print('train y rnn len:', len(rnn_list_y))
        print('test x rnn len:', len(rnn_list_test_x))
        print('test y rnn len:', len(rnn_list_test_y))

    

rnn_train_x = array(rnn_list_x)
rnn_train_y = array(rnn_list_y)
rnn_symbols = array(rnn_list_symbols)

rnn_test_x = array(rnn_list_test_x)
rnn_test_y = array(rnn_list_test_y)
rnn_test_symbols = array(rnn_list_test_symbols)


n_timesteps, n_features, n_outputs = rnn_train_x.shape[1], rnn_train_x.shape[2], rnn_train_y.shape[1]

# reshape output into [samples, timesteps, features]
rnn_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], rnn_train_y.shape[1], 1))

test_n_timesteps, test_n_features, test_n_outputs = rnn_test_x.shape[1], rnn_test_x.shape[2], rnn_test_y.shape[1]

# reshape output into [samples, timesteps, features]
rnn_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], rnn_test_y.shape[1], 1))


print('-'*20)
print('Overall totals')

print(' - train timesteps', n_timesteps)
print(' - train features', n_features)
print(' - train outputs', n_outputs)

print(' - train X shape: ', rnn_train_x.shape)
print(' - train y shape: ', rnn_train_y.shape)
print(' - train symbols lookup shape: ', rnn_symbols.shape)

print(' - test timesteps', test_n_timesteps)
print(' - test features', test_n_features)
print(' - test outputs', test_n_outputs)

print(' - test X shape: ', rnn_test_x.shape)
print(' - test y shape: ', rnn_test_y.shape)
print(' - test symbols lookup shape: ', rnn_test_symbols.shape)


/Users/neilkloot/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


--------------------------------------------------------------------------------
Symbol: VGS
 - train sequence shape: (8, 16, 1270)
 - test sequence shape: (1, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 8
train y rnn len: 8
test x rnn len: 1
test y rnn len: 1
--------------------------------------------------------------------------------
Symbol: ALB
 - train sequence shape: (12, 16, 1270)
 - test sequence shape: (2, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 20
train y rnn len: 20
test x rnn len: 3
test y rnn len: 3
--------------------------------------------------------------------------------
Symbol: CUA
 - train sequence shape: (5, 16, 1270)
 - test sequence shape: (1, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_tim

--------------------------------------------------------------------------------
Symbol: FPH
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 1008
train y rnn len: 1008
test x rnn len: 234
test y rnn len: 234
--------------------------------------------------------------------------------
Symbol: EWC
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 1068
train y rnn len: 1068
test x rnn len: 248
test y rnn len: 248
--------------------------------------------------------------------------------
Symbol: OTR
 - train sequence shape: (46, 16, 1270)
 - test sequence shape: (11, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_out

train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 2143
train y rnn len: 2143
test x rnn len: 499
test y rnn len: 499
--------------------------------------------------------------------------------
Symbol: IAU
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 2203
train y rnn len: 2203
test x rnn len: 513
test y rnn len: 513
--------------------------------------------------------------------------------
Symbol: CLV
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 2263
train y rnn len: 2263
test x rnn len: 527
test y rnn len: 527
--------------------------------

test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 3275
train y rnn len: 3275
test x rnn len: 765
test y rnn len: 765
--------------------------------------------------------------------------------
Symbol: NCK
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 3335
train y rnn len: 3335
test x rnn len: 779
test y rnn len: 779
--------------------------------------------------------------------------------
Symbol: DKM
 - train sequence shape: (15, 16, 1270)
 - test sequence shape: (3, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 3350
train y rnn len: 3350
test x rnn len: 782
test y rnn len: 782
-------------------------------------------------------------------------------

...

test n_outputs 1
-----
train x rnn len: 5387
train y rnn len: 5387
test x rnn len: 1263
test y rnn len: 1263
--------------------------------------------------------------------------------
Symbol: RXH
 - train sequence shape: (15, 16, 1270)
 - test sequence shape: (3, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 5402
train y rnn len: 5402
test x rnn len: 1266
test y rnn len: 1266
--------------------------------------------------------------------------------
Symbol: AAX
 - train sequence shape: (44, 16, 1270)
 - test sequence shape: (10, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 5446
train y rnn len: 5446
test x rnn len: 1276
test y rnn len: 1276
--------------------------------------------------------------------------------
Symbol: MMR
 - train sequence sha

test x rnn len: 1510
test y rnn len: 1510
--------------------------------------------------------------------------------
Symbol: GPP
 - train sequence shape: (56, 16, 1270)
 - test sequence shape: (13, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 6503
train y rnn len: 6503
test x rnn len: 1523
test y rnn len: 1523
--------------------------------------------------------------------------------
Symbol: AOP
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 6563
train y rnn len: 6563
test x rnn len: 1537
test y rnn len: 1537
--------------------------------------------------------------------------------
Symbol: TKL
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_

train x rnn len: 7371
train y rnn len: 7371
test x rnn len: 1724
test y rnn len: 1724
--------------------------------------------------------------------------------
Symbol: GLY
 - train sequence shape: (5, 16, 1270)
 - test sequence shape: (1, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 7376
train y rnn len: 7376
test x rnn len: 1725
test y rnn len: 1725
--------------------------------------------------------------------------------
Symbol: CNC
 - train sequence shape: (0,)
 - test sequence shape: (0,)
--------------------------------------------------------------------------------
Symbol: TPD
 - train sequence shape: (24, 16, 1270)
 - test sequence shape: (6, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 7400
train y rnn len: 7400
test x rnn len: 1731
test y r

--------------------------------------------------------------------------------
Symbol: FAC
 - train sequence shape: (26, 16, 1270)
 - test sequence shape: (6, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 8276
train y rnn len: 8276
test x rnn len: 1936
test y rnn len: 1936
--------------------------------------------------------------------------------
Symbol: SLE
 - train sequence shape: (24, 16, 1270)
 - test sequence shape: (6, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 8300
train y rnn len: 8300
test x rnn len: 1942
test y rnn len: 1942
--------------------------------------------------------------------------------
Symbol: KMC
 - train sequence shape: (24, 16, 1270)
 - test sequence shape: (6, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_ou

--------------------------------------------------------------------------------
Symbol: PAW
 - train sequence shape: (23, 16, 1270)
 - test sequence shape: (5, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 9070
train y rnn len: 9070
test x rnn len: 2119
test y rnn len: 2119
--------------------------------------------------------------------------------
Symbol: APD
 - train sequence shape: (60, 16, 1270)
 - test sequence shape: (14, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_outputs 1
-----
test n_timesteps 16
test n_features 1270
test n_outputs 1
-----
train x rnn len: 9130
train y rnn len: 9130
test x rnn len: 2133
test y rnn len: 2133
--------------------------------------------------------------------------------
Symbol: SDF
 - train sequence shape: (21, 16, 1270)
 - test sequence shape: (5, 16, 1270)
train n_timesteps 16
train n_features 1270
train n_o

In [ ]:
rnn_train_x

In [ ]:
# CNN 1D

from keras.models import Model, Sequential
from keras.layers import Input, Conv1D, Dense, Dropout, Lambda, concatenate, Flatten, MaxPooling1D
from keras import optimizers
from stats_operations import safe_log, safe_exp
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau


# convolutional layer parameters
n_filters = int(rnn_train_x.shape[2] / 2)
filter_width = 2
# dilation_rates = [2**i for i in range(8)] 
dilation_rates = [2**i for i in range(5)] 
batch_size = 2**11
epochs = 100

flat_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], 1))
flat_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], 1))


model = Sequential()
model.add(Conv1D(filters=n_filters, 
                 kernel_size=2, 
                 input_shape=(n_timesteps,n_features),
                 padding='causal',
                 dilation_rate=dilation_rates[0],
                 activation='relu'
                ))
model.add(MaxPooling1D(strides=2))

for dilation_rate in dilation_rates[1:-1]:
#     n_filters = int(n_filters/1.25)
#     print('filters', n_filters)
    model.add(Conv1D(filters=n_filters, 
                     kernel_size=2, 
                     padding='causal',
                     dilation_rate=dilation_rate,
                     activation='relu'
                    ))
    model.add(MaxPooling1D(strides=2))
    
# model.add(MaxPooling1D(strides=2))
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.summary()
model.compile(loss='mae', optimizer=optimizers.RMSprop(lr=0.005))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)

history = model.fit(rnn_train_x, safe_log(flat_train_y),
                    validation_data = (rnn_test_x, safe_log(flat_test_y)),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[reduce_lr],
                    verbose=1)

In [48]:
# CNN 2D

from keras.models import Model, Sequential
from keras.layers import Input, Conv1D, Dense, Dropout, Lambda, concatenate, Flatten, MaxPooling1D
from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers
from stats_operations import safe_log, safe_exp
from keras.callbacks import EarlyStopping, CSVLogger, ModelCheckpoint, ReduceLROnPlateau


# convolutional layer parameters
n_filters = 32 #int(rnn_train_x.shape[2] / 3)
filter_width = 2
# dilation_rates = [2**i for i in range(8)] 
# dilation_rates = [2**i for i in range(5)] 
dilation_rates = [2**i for i in range(3)] 
batch_size = 2**11
epochs = 100

cnn_2d_train_x = rnn_train_x.reshape((rnn_train_x.shape[0], rnn_train_x.shape[1], rnn_train_x.shape[2], 1))
cnn_2d_test_x = rnn_test_x.reshape((rnn_test_x.shape[0], rnn_test_x.shape[1], rnn_test_x.shape[2], 1))


flat_train_y = rnn_train_y.reshape((rnn_train_y.shape[0], 1))
flat_test_y = rnn_test_y.reshape((rnn_test_y.shape[0], 1))


model = Sequential()
model.add(Conv2D(filters=n_filters, 
                 kernel_size=(8,4), #(2,3), 
                 input_shape=(n_timesteps,n_features, 1),
#                  padding='causal',
                 dilation_rate=dilation_rates[0],
                 activation='relu'
                ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters=n_filters, 
                 kernel_size=(2,2), #(2,3), 
                 input_shape=(n_timesteps,n_features, 1),
#                  padding='causal',
                 dilation_rate=dilation_rates[0],
                 activation='relu'
                ))
model.add(MaxPooling2D(pool_size=(1,2)))


model.add(Conv2D(filters=n_filters, 
                 kernel_size=(2,2), #(2,3), 
                 dilation_rate=dilation_rates[1],
                 activation='relu'
                ))
model.add(MaxPooling2D(pool_size=(1,2)))

# model.add(Conv2D(filters=n_filters, 
#                  kernel_size=(2,2), #(2,3), 
#                  dilation_rate=dilation_rates[2],
#                  activation='relu'
#                 ))
# model.add(MaxPooling2D(pool_size=(1,2)))


# for dilation_rate in dilation_rates[1:-1]:
# #     n_filters = int(n_filters/1.25)
#     print('filters', n_filters)
#     model.add(Conv2D(filters=n_filters, 
#                      kernel_size=(2,2), #(2,3), 
# #                      padding='causal',
#                      dilation_rate=dilation_rate,
#                      activation='relu'
#                     ))
#     model.add(MaxPooling2D(strides=2))
    
model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.summary()
model.compile(loss='mae', optimizer=optimizers.Adam(lr=0.005))

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=1)
# early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(cnn_2d_train_x, safe_log(flat_train_y),
                    validation_data = (cnn_2d_test_x, safe_log(flat_test_y)),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[reduce_lr],
                    verbose=1)

# 32 filters: 1.8614 kernel (3,2)
# 16 filters: 1.8891 kernel (3,2)
# 48 filters: 1.8518 kernel (3,2)

# 32 filters: 1.8421 kernel (8,4) (2,2) (1,2)
# 32 filters: 1.8503 kernel (12,4) (3,4) (2,4) - No max pooling
# 32 filters: 1.8503 kernel (8,4) (3,2) (2,2) (2, 2) - max pooling (1, 2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_98 (Conv2D)           (None, 9, 1267, 32)       1056      
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 4, 633, 32)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 3, 632, 32)        4128      
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 3, 316, 32)        0         
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 1, 314, 32)        4128      
_________________________________________________________________
max_pooling2d_56 (MaxPooling (None, 1, 157, 32)        0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 5024)              0         
__________

9700/9700 [==============================] - 111s 11ms/step - loss: 2.0524 - val_loss: 1.8977
Epoch 60/100
9700/9700 [==============================] - 109s 11ms/step - loss: 2.0465 - val_loss: 1.8952
Epoch 61/100
9700/9700 [==============================] - 108s 11ms/step - loss: 2.0467 - val_loss: 1.8941
Epoch 62/100
9700/9700 [==============================] - 112s 12ms/step - loss: 2.0515 - val_loss: 1.8937
Epoch 63/100
9700/9700 [==============================] - 109s 11ms/step - loss: 2.0495 - val_loss: 1.8936
Epoch 64/100
9700/9700 [==============================] - 109s 11ms/step - loss: 2.0480 - val_loss: 1.8939
Epoch 65/100
9700/9700 [==============================] - 110s 11ms/step - loss: 2.0475 - val_loss: 1.8938
Epoch 66/100
9700/9700 [==============================] - 108s 11ms/step - loss: 2.0463 - val_loss: 1.8931
Epoch 67/100
9700/9700 [==============================] - 109s 11ms/step - loss: 2.0465 - val_loss: 1.8929
Epoch 68/100
9700/9700 [==========================

In [ ]:
safe_exp(np.array([1.8519]))

In [ ]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers


def DC_CNN_Block(nb_filter, filter_length, dilation, l2_layer_reg):
    def f(input_):
        
        residual =    input_
        
        layer_out =   Conv1D(filters=nb_filter, kernel_size=filter_length, 
                      dilation_rate=dilation, 
                      activation='linear', padding='causal', use_bias=False,
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(input_)
                    
        layer_out =   Activation('selu')(layer_out)
        
        skip_out =    Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
        
        network_in =  Conv1D(1,1, activation='linear', use_bias=False, 
                      kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, 
                      seed=42), kernel_regularizer=l2(l2_layer_reg))(layer_out)
                      
        network_out = Add()([residual, network_in])
        
        return network_out, skip_out
    
    return f

input = Input(shape=(n_timesteps,n_features))
    
l1a, l1b = DC_CNN_Block(128,2,1,0.001)(input)    
l2a, l2b = DC_CNN_Block(128,2,2,0.001)(l1a) 
l3a, l3b = DC_CNN_Block(128,2,4,0.001)(l2a)
l4a, l4b = DC_CNN_Block(128,2,8,0.001)(l3a)
l5a, l5b = DC_CNN_Block(128,2,16,0.001)(l4a)
l6a, l6b = DC_CNN_Block(128,2,32,0.001)(l5a)
l6b = Dropout(0.8)(l6b) #dropout used to limit influence of earlier data
l7a, l7b = DC_CNN_Block(128,2,64,0.001)(l6a)
l7b = Dropout(0.8)(l7b) #dropout used to limit influence of earlier data

l8 =   Add()([l1b, l2b, l3b, l4b, l5b, l6b, l7b])

l9 =   Activation('relu')(l8)

# l21 =  Conv1D(1,1, activation='linear', use_bias=False, 
#        kernel_initializer=TruncatedNormal(mean=0.0, stddev=0.05, seed=42),
#        kernel_regularizer=l2(0.001))(l9)
l22 = Flatten()(l9)
l23 = Dense(1)(l22)

model = Model(input=input, output=l23)
model.summary()

adam = optimizers.Adam(lr=0.00075, beta_1=0.9, beta_2=0.999, epsilon=None, 
                       decay=0.0, amsgrad=False)

model.compile(loss='mae', optimizer=adam, metrics=['mse'])

history = model.fit(rnn_train_x, safe_log(flat_train_y),
                    validation_data = (rnn_test_x, safe_log(flat_test_y)),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[reduce_lr],
                    verbose=1)